# Changing to Parent Directory to Import Needed Libraries

In [15]:
import os

# Change this relative to your own directory structure
PARENT_DIR = '/Users/henrygilbert/GitHub/CS-6362/final_project'
os.chdir(PARENT_DIR)

# Importing Needed Libraries

In [16]:
import torch
import numpy as np
import pandas as pd
import importlib
import copy
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import mlflow

import utilities
import factor_data_loader
import market_data_loader
from model import CVAE


# optional reload for libraries if needed
importlib.reload(utilities)
mlflow.set_experiment("Experiment 4")

mlflow.end_run()
mlflow.start_run()

<ActiveRun: >

# Conditional Synthetic Data Generation Data Loading

In [17]:


def create_monthly_factor_conditioned_data(start_ts: pd.Timestamp, end_ts: pd.Timestamp, factor: factor_data_loader.Factor):
    
    mdl = market_data_loader.MarketDataLoader()
    fdl = factor_data_loader.FactorDataLoader()

    # Load SPY Price data
    monthly_eod_prices = mdl.get_eod_price_data_grouped('SPY', start_ts, end_ts, market_data_loader.GroupPeriod.MONTHLY)
    monthly_percent_change_prices = {k: np.diff(v)/v[:-1] for k, v in monthly_eod_prices.items()}
    monthly_means_to_std = {np.mean(v): np.std(v) for k, v in monthly_percent_change_prices.items()}

    mean_price_change = np.mean(np.nan_to_num(list(monthly_means_to_std.keys())))
    std_price_change = np.mean(np.nan_to_num(list(monthly_means_to_std.values())))

    week_size = 5
    weekly_data = {k: np.array([v[i:i+week_size] for i in range(0, len(v), week_size)]) for k, v in monthly_percent_change_prices.items()}

    # Load Factor Data
    factor_data_by_month = fdl.get_factor_data_by_month(factor, start_ts, end_ts)

    # Removes auto-correlation in the data - get's rid of first month
    factor_months = list(factor_data_by_month.keys())
    factor_month_data = np.array(list(factor_data_by_month.values()))
    
    percent_diff = np.diff(factor_month_data)/factor_month_data[:-1]
    percent_diff[percent_diff == -np.inf] = 0
    percent_diff[percent_diff == np.inf] = 0
    percent_diff = np.nan_to_num(percent_diff)
    factor_months.pop(0)
    
    mean_factor_change = np.mean(percent_diff)
    std_factor_change = np.std(percent_diff)
    
    assert len(percent_diff) == len(factor_months)
    factor_data_by_month = {factor_months[i]: percent_diff[i] for i in range(len(factor_months))}

    weekly_data = {k: v for k, v in weekly_data.items() if k in factor_data_by_month}
    weekly_factor_conditioned_training_data = []

    for month, weekly_prices in weekly_data.items():
        
        factor_values = factor_data_by_month[month]
        for i in range(1, len(weekly_prices)):
            
            if len(weekly_prices[i]) != 5 or len(weekly_prices[i-1]) != 5:
                continue

            factor_conditioning_data = np.concatenate((np.array([factor_values]), weekly_prices[i-1]))
            weekly_factor_conditioned_training_data.append((factor_conditioning_data, weekly_prices[i]))

    month_batch_size = 4
    monthly_factor_conditioned_batches = [
        weekly_factor_conditioned_training_data[i:i+month_batch_size] 
        for i in range(0, len(weekly_factor_conditioned_training_data), month_batch_size)]
    
    return monthly_factor_conditioned_batches, mean_price_change, std_price_change, mean_factor_change, std_factor_change


# Conditional Evaluation

In [18]:
batch_size = 4
batch_to_rmse = {}
synthetic_means = []
synthetic_stds = []
mlflow.log_param("training_batch_size", batch_size)

for factor in factor_data_loader.Factor:
    
    start_ts = pd.Timestamp('2016-01-01')
    end_ts = pd.Timestamp('2021-02-01')
    monthly_factor_conditioned_batches, mean_price_change, std_price_change, mean_factor_change, std_factor_change = create_monthly_factor_conditioned_data(start_ts, end_ts, factor)

    split_index = int(len(monthly_factor_conditioned_batches)*0.8)
    monthly_factor_conditioned_training_data = monthly_factor_conditioned_batches
    monthly_factor_conditioned_testing_data = monthly_factor_conditioned_batches[split_index:]

    factor_conditioned_training_weeks = [week for batch in monthly_factor_conditioned_training_data for week in batch]
    factor_conditioned_testing_weeks = [week for batch in monthly_factor_conditioned_testing_data for week in batch]

    factor_training_data = utilities.ConditionedMarketDataset(factor_conditioned_training_weeks)
    factor_testing_data = utilities.ConditionedMarketDataset(factor_conditioned_testing_weeks)

    factor_train_dataset = DataLoader(factor_training_data, batch_size=batch_size, shuffle=True)
    factor_testing_dataset = DataLoader(factor_testing_data, batch_size=batch_size, shuffle=True)

    factor_cvae = CVAE(5, 6).to(utilities.DEVICE)
    factor_history = utilities.train_model(factor_cvae, factor_train_dataset, factor_testing_dataset, epochs=10)
    print(factor_history)
    [mlflow.log_metric(f"factor_validation_loss", val_loss) for val_loss in factor_history]
    
    synthetic_mean_val_base_returns = []
    synthetic_val_base_returns = []
    
    synthetic_mean_val_large_returns = []
    synthetic_val_large_returns = []
    
    synthetic_mean_val_small_returns = []
    synthetic_val_small_returns = []
    
    control_mean_val_returns = []    
    actual_val_returns = []
    num_synthetic_samples = 1000
    mlflow.log_param("num_synthetic_samples", num_synthetic_samples)
    for batch in factor_testing_dataset:
        
        price_batch = batch['price_data']
        conditioned_batch = batch['factor_data']        
        
        synthetic_base_factor_batches = [
            torch.FloatTensor(
                np.array([
                    np.concatenate((
                        np.random.normal(loc=mean_factor_change, scale=std_factor_change, size=1), 
                        batch[-5:])) 
                    for batch in conditioned_batch])) 
            for _ in range(num_synthetic_samples)]
        synthetic_large_factor_batches = [
            torch.FloatTensor(
                np.array([
                    np.concatenate((
                        np.random.normal(loc=mean_factor_change+std_factor_change, scale=std_factor_change, size=1), 
                        batch[-5:])) 
                    for batch in price_batch])) 
            for _ in range(num_synthetic_samples)]
        synthetic_small_factor_batches = [
            torch.FloatTensor(
                np.array([
                    np.concatenate((
                        np.random.normal(loc=mean_factor_change-std_factor_change, scale=std_factor_change, size=1), 
                        batch[-5:])) 
                    for batch in price_batch])) 
            for _ in range(num_synthetic_samples)]
        
        
        price_batch = price_batch.to(utilities.DEVICE)
       
        # This is wrong, should be generating synthetic conditional data not price you dummy
        sample_synthetic_base_returns = [
            factor_cvae(price_batch.float(), synthetic_factors.float()).detach().numpy() 
            for synthetic_factors in synthetic_base_factor_batches]
        sample_synthetic_large_returns = [
            factor_cvae(price_batch.float(), synthetic_factors.float()).detach().numpy() 
            for synthetic_factors in synthetic_large_factor_batches]
        sample_synthetic_small_returns = [
            factor_cvae(price_batch.float(), synthetic_factors.float()).detach().numpy() 
            for synthetic_factors in synthetic_small_factor_batches]
        
        mean_synthetic_base_returns = np.mean(sample_synthetic_base_returns, axis=0)
        mean_synthetic_large_returns = np.mean(sample_synthetic_large_returns, axis=0)
        mean_synthetic_small_returns = np.mean(sample_synthetic_small_returns, axis=0)
        mean_control_returns = factor_cvae(price_batch.float(), conditioned_batch.float()).detach().numpy()
        actual_val_returns += list(price_batch.detach().numpy())
        
        print(f"Given price: {price_batch[0]}")
        print(f"Computed price: {mean_control_returns[0]}\n")
        
        synthetic_val_base_returns += sample_synthetic_base_returns
        synthetic_mean_val_base_returns += list(mean_synthetic_base_returns)
        
        synthetic_val_large_returns += sample_synthetic_large_returns
        synthetic_mean_val_large_returns += list(mean_synthetic_large_returns)
        
        synthetic_val_small_returns += sample_synthetic_small_returns
        synthetic_mean_val_small_returns += list(mean_synthetic_small_returns)

        control_mean_val_returns += list(mean_control_returns)
        
    synthetic_mean_val_base_returns = np.array(synthetic_mean_val_base_returns).flatten()
    synthetic_mean_val_large_returns = np.array(synthetic_mean_val_large_returns).flatten()
    synthetic_mean_val_small_returns = np.array(synthetic_mean_val_small_returns).flatten()
    
    control_mean_val_returns = np.array(control_mean_val_returns).flatten()
    actual_val_returns = np.array(actual_val_returns).flatten()
    
    base_mean_rmse = np.sqrt(np.mean((synthetic_mean_val_base_returns - actual_val_returns)**2))
    base_std_rmse = np.sqrt(np.mean((np.std(synthetic_mean_val_base_returns) - np.std(actual_val_returns))**2))
    
    large_mean_rmse = np.sqrt(np.mean((synthetic_mean_val_large_returns - actual_val_returns)**2))
    large_std_rmse = np.sqrt(np.mean((np.std(synthetic_mean_val_large_returns) - np.std(actual_val_returns))**2))
    
    small_mean_rmse = np.sqrt(np.mean((synthetic_mean_val_small_returns - actual_val_returns)**2))
    small_std_rmse = np.sqrt(np.mean((np.std(synthetic_mean_val_small_returns) - np.std(actual_val_returns))**2))
    
    mlflow.log_metric(f"base_rmse", base_mean_rmse + base_std_rmse)
    mlflow.log_metric(f"large_rmse", large_mean_rmse + large_std_rmse)
    mlflow.log_metric(f"small_rmse", small_mean_rmse + small_std_rmse)
    
    plt.plot(synthetic_mean_val_base_returns, label=f"Synthetic Base Std: {np.std(synthetic_mean_val_base_returns)}")
    plt.plot(synthetic_mean_val_small_returns, label=f"Synthetic Small Std: {np.std(synthetic_mean_val_small_returns)}")
    plt.plot(synthetic_mean_val_large_returns, label=f"Synthetic Large Std: {np.std(synthetic_mean_val_large_returns)}")
    plt.legend()
    plt.xlabel("week number")
    plt.ylabel("price in USD")
    plt.title(f"Learned Factor Relationships: {factor.value}")
    plt.savefig(f"experiments/experiment_4/graphs/synthetic_base_returns_{factor.value}.png")
    mlflow.log_artifact(f"experiments/experiment_4/graphs/synthetic_base_returns_{factor.value}.png")
    plt.clf()
    plt.close()
  
    
    

mlflow.end_run()

/Users/henrygilbert/GitHub/CS-6362/final_project/market_data_loader.py:66: UserWarning: Converting to Period representation will drop timezone information.
  group.index[0].to_period(group_by.value).to_timestamp(): group['close'].to_numpy()
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid valu

[0.12734292447566986, 0.092103973031044, 0.07123871892690659, 0.060816287994384766, 0.04762667044997215, 0.039057470858097076, 0.03553412854671478, 0.03213329240679741, 0.027407817542552948, 0.022590216249227524]
Given price: tensor([ 0.0036,  0.0101,  0.0021,  0.0064, -0.0021], dtype=torch.float64)
Computed price: [ 0.00326191 -0.02291307  0.00919171 -0.02026137  0.00330888]

Given price: tensor([ 0.0090,  0.0162, -0.0062, -0.0064, -0.0011], dtype=torch.float64)
Computed price: [ 0.03680572 -0.01513638 -0.00469192 -0.04740755 -0.00839506]

Given price: tensor([ 0.0438, -0.0491, -0.0693,  0.0586, -0.1078], dtype=torch.float64)
Computed price: [ 0.02161523 -0.00798525 -0.0226056   0.00387289  0.01349523]

Given price: tensor([ 0.0139,  0.0149, -0.0049,  0.0261, -0.0086], dtype=torch.float64)
Computed price: [-0.00561631 -0.02544484  0.00815344 -0.04631897 -0.04953275]

Given price: tensor([ 0.0097, -0.0089,  0.0131,  0.0096, -0.0035], dtype=torch.float64)
Computed price: [ 0.01100323 -0

/Users/henrygilbert/GitHub/CS-6362/final_project/market_data_loader.py:66: UserWarning: Converting to Period representation will drop timezone information.
  group.index[0].to_period(group_by.value).to_timestamp(): group['close'].to_numpy()
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid valu

[0.11840997636318207, 0.08507055044174194, 0.06693588197231293, 0.055529024451971054, 0.04805232584476471, 0.040650587528944016, 0.03415969759225845, 0.027808304876089096, 0.025835542008280754, 0.021548431366682053]
Given price: tensor([ 0.0028, -0.0180, -0.0032, -0.0344,  0.0103], dtype=torch.float64)
Computed price: [ 0.00811022  0.00016182  0.01198175  0.01974734 -0.01696028]

Given price: tensor([-0.0049, -0.0127,  0.0048, -0.0073,  0.0061], dtype=torch.float64)
Computed price: [-0.02938459  0.00495127 -0.02319552 -0.00824814 -0.00850004]

Given price: tensor([-0.0082,  0.0135, -0.0015,  0.0001,  0.0032], dtype=torch.float64)
Computed price: [-0.01954359 -0.0090304   0.00631192  0.01346172  0.0097851 ]

Given price: tensor([ 0.0184, -0.0035,  0.0003, -0.0103,  0.0069], dtype=torch.float64)
Computed price: [ 9.5332041e-05 -1.6451128e-02 -1.4387228e-02  4.0444378e-03
  6.1928853e-04]

Given price: tensor([ 0.0014,  0.0058, -0.0082, -0.0041, -0.0010], dtype=torch.float64)
Computed pri

/Users/henrygilbert/GitHub/CS-6362/final_project/market_data_loader.py:66: UserWarning: Converting to Period representation will drop timezone information.
  group.index[0].to_period(group_by.value).to_timestamp(): group['close'].to_numpy()
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid valu

...

[0.1258864402770996, 0.08348184078931808, 0.06384367495775223, 0.06142487749457359, 0.05450582876801491, 0.049890004098415375, 0.03574223816394806, 0.03475389629602432, 0.031830139458179474, 0.030511008575558662]
Given price: tensor([ 0.0020, -0.0043,  0.0035,  0.0034,  0.0102], dtype=torch.float64)
Computed price: [ 0.00697002  0.00534121  0.03255958 -0.01645537 -0.01970352]

Given price: tensor([-0.0160,  0.0104, -0.0008,  0.0031, -0.0179], dtype=torch.float64)
Computed price: [ 0.03191499  0.03785421  0.01147036 -0.02503968 -0.02021581]

Given price: tensor([ 0.0016,  0.0065, -0.0008,  0.0013, -0.0026], dtype=torch.float64)
Computed price: [ 0.01247438  0.01737285  0.01747572 -0.01593738  0.00720171]

Given price: tensor([-0.0015,  0.0075, -0.0095,  0.0135,  0.0125], dtype=torch.float64)
Computed price: [-0.01072378 -0.00159712 -0.06342107 -0.01543164  0.00310585]

Given price: tensor([ 0.0004, -0.0207, -0.0177,  0.0126,  0.0044], dtype=torch.float64)
Computed price: [-0.02485116  0

/Users/henrygilbert/GitHub/CS-6362/final_project/market_data_loader.py:66: UserWarning: Converting to Period representation will drop timezone information.
  group.index[0].to_period(group_by.value).to_timestamp(): group['close'].to_numpy()
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid valu

[0.10696432739496231, 0.08247711509466171, 0.06712199002504349, 0.05409538745880127, 0.05140630900859833, 0.044878676533699036, 0.04125003516674042, 0.029408050701022148, 0.03531793877482414, 0.022461723536252975]
Given price: tensor([ 0.0028, -0.0180, -0.0032, -0.0344,  0.0103], dtype=torch.float64)
Computed price: [-0.02462307 -0.03791229  0.0057666  -0.00019446  0.01103939]

Given price: tensor([ 0.0302, -0.0093,  0.0165, -0.0074,  0.0022], dtype=torch.float64)
Computed price: [-0.00644134 -0.00402476 -0.01983562 -0.00192946  0.01900487]

Given price: tensor([ 0.0047, -0.0039, -0.0105, -0.0331, -0.0305], dtype=torch.float64)
Computed price: [-0.00434864  0.0073993   0.00829912  0.01132883 -0.0045137 ]

Given price: tensor([ 0.0043, -0.0250,  0.0100, -0.0233,  0.0149], dtype=torch.float64)
Computed price: [ 0.0152988  -0.0291358  -0.03060339  0.00141728  0.0378506 ]

Given price: tensor([ 0.0005,  0.0042,  0.0086, -0.0016,  0.0011], dtype=torch.float64)
Computed price: [ 0.02333215 -

/Users/henrygilbert/GitHub/CS-6362/final_project/market_data_loader.py:66: UserWarning: Converting to Period representation will drop timezone information.
  group.index[0].to_period(group_by.value).to_timestamp(): group['close'].to_numpy()
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/homebrew/Caskroom/miniforge/base/envs/cs-6362/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid valu

[0.12406650930643082, 0.07942719757556915, 0.06398231536149979, 0.05339803174138069, 0.045506421476602554, 0.03608871251344681, 0.035864897072315216, 0.030948877334594727, 0.02751552127301693, 0.021945640444755554]
Given price: tensor([ 0.0544, -0.0728,  0.0206, -0.0464, -0.0293], dtype=torch.float64)
Computed price: [ 0.01236297 -0.00997932 -0.00952565  0.00922907  0.00236576]

Given price: tensor([-0.0006, -0.0149,  0.0034, -0.0019,  0.0058], dtype=torch.float64)
Computed price: [ 0.00055058  0.00823701  0.01091497 -0.00359972  0.0022176 ]

Given price: tensor([-0.0001,  0.0027, -0.0029, -0.0077,  0.0079], dtype=torch.float64)
Computed price: [ 0.02917682 -0.02546582  0.01869876  0.00339079  0.01464287]

Given price: tensor([ 0.0027,  0.0164,  0.0168, -0.0054,  0.0057], dtype=torch.float64)
Computed price: [-0.01608825 -0.01771603  0.00027867  0.00741395  0.00995399]

Given price: tensor([ 0.0026,  0.0080,  0.0019,  0.0059, -0.0118], dtype=torch.float64)
Computed price: [ 0.02715746 